In [ ]:
import os
import geopandas as gpd
import pandas as pd

In [ ]:
# Read the modified catchment shapefile
modifiedcat = gpd.read_file('/home/paulc600/SMM/SMM HYPE files/Modified_SMMcat.shp')

In [ ]:
# Read the river shapefile
riv = gpd.read_file('/home/paulc600/github/StMaryMilk2023-UofC/modified_TGF/smm_tgf_modified/smm_riv.shp')

In [ ]:
# Set the output folder path where you want to save the text files
output_folder_path = '/home/paulc600/local/HYPE Inputs/'

In [ ]:
# Sort the GeoDataFrame by a specific column
sorted_modifiedcat = modifiedcat.sort_values(by='seg_nhm')
sorted_riv = riv.sort_values(by='seg_nhm')

In [ ]:
sorted_modifiedcat

In [ ]:
sorted_riv

In [ ]:
# Select the specific rows based on the range in a column
selected_rows = modifiedcat[(modifiedcat['seg_nhm'] >= 58662 )]

In [ ]:
# Concatenate the selected rows with the river shapefile
merged_data = pd.concat([riv, selected_rows], ignore_index=True)

In [ ]:
# Convert the merged data to a GeoDataFrame
merged_riv = gpd.GeoDataFrame(merged_data, crs=riv.crs)

In [ ]:
# fill na values
merged_riv=merged_riv.fillna(0)

In [ ]:
merged_riv

Start to create GeoData.txt

In [ ]:
merged_riv

In [ ]:
# Extract the first two columns from the merged_riv GeoDataFrame (subid and maindown)
geodata = merged_riv.iloc[:, :2]

In [ ]:
# Perform the merge based on a common column
geodata = pd.merge(geodata, modifiedcat[['seg_nhm', 'Shape_Area']], on='seg_nhm')

Add Lat and long grom statistics in gistool (optional)

Add elev_mean from Merit Hydro in gistool with the smm_cat.shp (optional)

In [ ]:
# merge slope to the dataframe from merged_riv
geodata = pd.merge(geodata, merged_riv[['seg_nhm', 'seg_slope']], on='seg_nhm')

In [ ]:
# merge length to the dataframe from merged_riv
geodata = pd.merge(geodata, merged_riv[['seg_nhm', 'Shape_Leng']], on='seg_nhm')

In [ ]:
geodata

In [ ]:
# read the slc information
slc = pd.read_csv('/home/paulc600/SMM/HYPE_geospatial/HYPE_output.csv')

In [ ]:
slc

In [ ]:
# Merge the data from the CSV file with modifiedcat matching the hru ids
merged_cat = pd.merge(modifiedcat, slc, left_on='hru_nhm', right_on='Unnamed: 0', how='inner')

In [ ]:
merged_cat

In [ ]:
# Get the columns to merge based on a range or condition
column_range = merged_cat.columns[14:]  

In [ ]:
column_range

In [ ]:
# Merge the selected columns from merged_data with data_to_write matching seg ids
final_geodata = pd.merge(geodata, merged_cat[['seg_nhm'] + list(column_range)],  on='seg_nhm')
final_geodata

In [ ]:
# changing downstream values of nca
final_geodata.loc[final_geodata['seg_nhm'] >= 58662, 'ds_seg_nhm'] = -9999

In [ ]:
# zero the rest of the columns for nca
final_geodata.loc[final_geodata['seg_nhm'] >= 58662, 'Shape_Area':'Shape_Leng'] = 0

In [ ]:
final_geodata

In [ ]:
#Rename the first columns to HYPE GeoData format
final_geodata.rename(columns={'seg_nhm':'subid' , 'ds_seg_nhm':'maindown', 'Shape_Area':'area', 'seg_slope':'slope_mean', 'Shape_Leng':'rivlen'}, inplace=True)   

In [ ]:
# Define the range of columns to rename
start_index = 5
end_index = 76
offset=start_index-1

In [ ]:
# Generate the new column names
new_column_names = ['SLC_' + str(i - offset) for i in range(start_index, end_index + 1)]

In [ ]:
new_column_names

In [ ]:
# Rename the columns in merged_data
final_geodata.columns.values[start_index:end_index + 1] = new_column_names

In [ ]:
final_geodata

In [ ]:
# Create a custom sorting key based on 'upstream' and 'downstream' columns
final_geodata['sorting_key'] = final_geodata.apply(lambda row: (row['subid'], row.name)
                                       if row['subid'] not in final_geodata['maindown'].values
                                       else (float('inf'), row.name), axis=1)


In [ ]:
# Sort the DataFrame from upstream to downstream
geodata_sorted = final_geodata.sort_values('sorting_key', ascending=False)

In [ ]:
# Remove the sorting key column
geodata_sorted = geodata_sorted.drop('sorting_key', axis=1)

In [ ]:
geodata_sorted

In [ ]:
geodata_sorted.to_csv(os.path.join(output_folder_path,'GeoData.txt'), sep='\t', index=False)